In [5]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os

def cropVideo(src, dest, x,y,h,w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest, threads=4)

# cropVideo(r"C:\Users\Dell\Downloads\Did you know Obsidian isn't just a rock It's a glimpse into our volcanic past! 🔥 #DidYouKnow #ObsidianFacts #Geology101.mp4", r"media\Did you know Obsidian isn't just a rock It's a glimpse into our volcanic past! 🔥 #DidYouKnow #ObsidianFacts #Geology101.mp4", 69,244,1676,942)
title = "Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth"
title.replace(": ", " ")
downloadName = "Do you know- Cholera-1713704344"
recent_mp4 = os.path.join(os.path.expanduser("~/Downloads"), downloadName + ".mp4")

current_directory = os.path.join(os.getcwd(), "media")
path = os.path.join(current_directory, title + ".mp4")
cropVideo(recent_mp4, path, 69, 244, 1676, 942)
os.remove(recent_mp4)

Moviepy - Building video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth.mp4.
MoviePy - Writing audio in Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealthTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth.mp4



Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth.mp4


In [3]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os
import threading

def cropVideo(src, dest, x, y, h, w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest, threads=4)

def process_video(title_info):
    title = title_info['title']
    downloadName = title_info['download_name']
    recent_mp4 = os.path.join(os.path.expanduser("~/Downloads"), downloadName + ".mp4")
    
    current_directory = os.path.join(os.getcwd(), "media")
    dest_path = os.path.join(current_directory, title + ".mp4")
    
    cropVideo(recent_mp4, dest_path, title_info['x'], title_info['y'], title_info['h'], title_info['w'])
    os.remove(recent_mp4)

# Read titles from a file (assuming each line contains title info in the format: title,downloadName,x,y,h,w)
titles_info = []
with open("titles.txt", "r", encoding="utf-8") as f:
    for line in f.read().split("\n"):
        if line != "":
            title_info = line.split(":")
            title_info = [i.removeprefix(" ").removesuffix(" ") for i in title_info]
            titles_info.append({
                'title': title_info[0],
                'download_name': title_info[1],
                'x': 69,
                'y': 244,
                'h': 1676,
                'w': 942
            })

for title_info in titles_info:
    print(title_info)
    process_video(title_info)

{'title': "The Burj Khalifa is the world's tallest building with 163 floors. Have you seen its stunning design #DidYouKnow #BurjKhalifa", 'download_name': 'Burj Khalifa-1713979408', 'x': 69, 'y': 244, 'h': 1676, 'w': 942}
Moviepy - Building video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\The Burj Khalifa is the world's tallest building with 163 floors. Have you seen its stunning design #DidYouKnow #BurjKhalifa.mp4.
MoviePy - Writing audio in The Burj Khalifa is the world's tallest building with 163 floors. Have you seen its stunning design #DidYouKnow #BurjKhalifaTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\The Burj Khalifa is the world's tallest building with 163 floors. Have you seen its stunning design #DidYouKnow #BurjKhalifa.mp4



Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\The Burj Khalifa is the world's tallest building with 163 floors. Have you seen its stunning design #DidYouKnow #BurjKhalifa.mp4
{'title': "Viruses aren't technically alive but can cause major diseases. Learn about these tiny agents! #DidYouKnow #Viruses #ScienceFacts", 'download_name': 'Viruses-1713979401', 'x': 69, 'y': 244, 'h': 1676, 'w': 942}
Moviepy - Building video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Viruses aren't technically alive but can cause major diseases. Learn about these tiny agents! #DidYouKnow #Viruses #ScienceFacts.mp4.
MoviePy - Writing audio in Viruses aren't technically alive but can cause major diseases. Learn about these tiny agents! #DidYouKnow #Viruses #ScienceFactsTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Viruses aren't technically alive but can cause major diseases. Learn about these tiny agents! #DidYouKnow #Viruses #ScienceFacts.mp4



Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Viruses aren't technically alive but can cause major diseases. Learn about these tiny agents! #DidYouKnow #Viruses #ScienceFacts.mp4


In [2]:
title_info

['']